### AI Code Explainer plus Language Translator
Technical question answering bot for code explaination and related queries plus language translator for the english output.

In [1]:
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
import os

d:\MyWorkD\MyProject\tfvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Initializing

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

openai_ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# Prompts

system_prompt = """You are an AI assistant and code expert that explains in brief within 300 words what certain lines of code does and why its written in that manner. \
                   Also point out if there is anything wrong with the code and provide the corrected code."""
user_prompt= "Provide a short and detailed explaination on what the given code does and why :"

In [ ]:
# Functions for chat interface.

# Using GPT mode
def get_ans_gpt(code):
    stream = openai.chat.completions.create(
        model= MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt+f"\n{code}"}
      ],
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# Using open source gemma model in ollama
def get_ans_gemma(code):
    stream = openai_ollama.chat.completions.create(
        model='gemma3:1b',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt+f"\n{code}"}
      ],
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# Providing user the option to choose desired model.
def stream_model(question, model):
    if model== "gpt":
        output= get_ans_gpt(question)
    elif model== "gemma":
        output= get_ans_gemma(question)
    else:
        raise ValueError('Unknown model')
    yield from output

In [5]:
# User interface using Gradio

ui= gr.Interface(
    fn= stream_model,
    inputs=[gr.Textbox(label="Your code to explain"), gr.Dropdown(['llama','gemma'], label= "Select model", value="llama")],
    outputs=[gr.Markdown(label="Explaination")],
    flagging_mode= "never"
)

In [10]:
ui.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


### Translate the chat to a different language

In [ ]:
# Function for language translation with prompts

def translate(msg,lang):

    response = openai.chat.completions.create(model=MODEL,
                                                     messages=[
                {"role": "system", "content":"You are a text translater that converts english texts to french and respond with only the converted text, nothing else."},
                {"role": "user", "content": "convert the following text to "+f"{lang}"+f" :" +f"\n{msg}"}
                ])
    
    return  response.choices[0].message.content

In [ ]:
# User interface with gradio

with gr.Blocks() as ui:

    with gr.Row():
        code= gr.Textbox(label="Enter your code here", lines=5.5, min_width=300)
        with gr.Column():
            selected_mdl= gr.Dropdown(['gpt','gemma'], label= "Select Model", value="gemma")
            selected_lang= gr.Dropdown(['french','german','mandarin','korean'], label= "Select Language", value="french")

    with gr.Row():
        submit= gr.Button('Submit')
        translate_btn= gr.Button('Translate')

    with gr.Row():
        explained= gr.TextArea(label="Explaination:", lines=8)
        translated= gr.TextArea(label="Translated Response:", lines=8)
    
    with gr.Row():
        clear = gr.Button("Clear")

    submit.click(stream_model, inputs=[code, selected_mdl], outputs=[explained])
    translate_btn.click(translate, inputs=[explained, selected_lang], outputs=[translated])
    clear.click(lambda: [None,None,None], inputs=None, outputs=[code,explained,translated], queue=False)

ui.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
